# Etapa 1

In [5]:
from pyspark.sql.session import SparkSession
from pyspark import SparkContext, SQLContext

spark = SparkSession.builder.master("local[*]").appName("Ex2").getOrCreate()

names_df = spark.read.csv("nomes_aleatorios.txt")

names_df.show(5)

+---------------+
|            _c0|
+---------------+
| Martin Nattiah|
|  Maryann Mcgee|
|    Kyle Keaton|
|Frank Boudreaux|
|  Justin Walton|
+---------------+
only showing top 5 rows



# Etapa 2

In [7]:
names_df = names_df.withColumnRenamed("_c0", "Nomes")

names_df.printSchema()
names_df.show(10)

root
 |-- Nomes: string (nullable = true)

+---------------+
|          Nomes|
+---------------+
| Martin Nattiah|
|  Maryann Mcgee|
|    Kyle Keaton|
|Frank Boudreaux|
|  Justin Walton|
|Tiffany Parnell|
|     Ralph Gray|
|    John Rusnak|
|   Nelson Bland|
|   Michael Levi|
+---------------+
only showing top 10 rows



# Etapa 3

In [17]:
from pyspark.sql import functions as dff
from random import choice

df_choice = dff.udf(lambda: choice(["Fundamental", "Medio", "Superior"]))
education_df = names_df.withColumn("Escolaridade", df_choice())

education_df.show(10)

+---------------+------------+
|          Nomes|Escolaridade|
+---------------+------------+
| Martin Nattiah| Fundamental|
|  Maryann Mcgee|    Superior|
|    Kyle Keaton|       Medio|
|Frank Boudreaux| Fundamental|
|  Justin Walton| Fundamental|
|Tiffany Parnell| Fundamental|
|     Ralph Gray| Fundamental|
|    John Rusnak|    Superior|
|   Nelson Bland|       Medio|
|   Michael Levi|       Medio|
+---------------+------------+
only showing top 10 rows



# Etapa 4

In [23]:
south_american_countries = ["Argentina", "Bolivia", "Brazil", "Chile", "Colombia", "French Guiana", "Ecuador", "Guyana", "Paraguay", "Peru", "Suriname", "Uruguay", "Venezuela"]

dff_choice = dff.udf(lambda: choice(south_american_countries))
country_df = education_df.withColumn("Pais", dff_choice())

country_df.show(5)

+---------------+------------+---------+
|          Nomes|Escolaridade|     Pais|
+---------------+------------+---------+
| Martin Nattiah| Fundamental|Argentina|
|  Maryann Mcgee|    Superior| Paraguay|
|    Kyle Keaton|       Medio|  Ecuador|
|Frank Boudreaux| Fundamental|  Bolivia|
|  Justin Walton| Fundamental|  Bolivia|
+---------------+------------+---------+
only showing top 5 rows



# Etapa 5

In [24]:
from pyspark.sql.types import IntegerType
from random import randint

dff_randint = dff.udf(lambda: randint(1945, 2010), IntegerType())
birth_date_df = country_df.withColumn("AnoNascimento", dff_randint())

birth_date_df.show(5)

+---------------+------------+---------+-------------+
|          Nomes|Escolaridade|     Pais|AnoNascimento|
+---------------+------------+---------+-------------+
| Martin Nattiah| Fundamental|Argentina|         1952|
|  Maryann Mcgee|    Superior| Paraguay|         1998|
|    Kyle Keaton|       Medio|  Ecuador|         1957|
|Frank Boudreaux| Fundamental|  Bolivia|         1953|
|  Justin Walton| Fundamental|  Bolivia|         1956|
+---------------+------------+---------+-------------+
only showing top 5 rows



# Etapa 6

In [26]:
dff_get_century = dff.udf(lambda n: (n // 100) + 1, IntegerType())

select_df = birth_date_df.select("*").where(dff_get_century(dff.col("AnoNascimento")) == 21)

select_df.show(5)

+----------------+------------+---------+-------------+
|           Nomes|Escolaridade|     Pais|AnoNascimento|
+----------------+------------+---------+-------------+
|      Ralph Gray| Fundamental|Argentina|         1952|
|    Nelson Bland|    Superior| Paraguay|         1998|
|     Jack Worden|       Medio|  Ecuador|         1957|
|Benjamin Schmidt| Fundamental|  Bolivia|         1953|
|    Thomas Price| Fundamental|  Bolivia|         1956|
+----------------+------------+---------+-------------+
only showing top 5 rows



# Etapa 7

In [27]:
birth_date_df.createOrReplaceTempView("People")

spark.sql("SELECT * FROM People").show()

+-----------------+------------+-------------+-------------+
|            Nomes|Escolaridade|         Pais|AnoNascimento|
+-----------------+------------+-------------+-------------+
|   Martin Nattiah| Fundamental|    Argentina|         1952|
|    Maryann Mcgee|    Superior|     Paraguay|         1998|
|      Kyle Keaton|       Medio|      Ecuador|         1957|
|  Frank Boudreaux| Fundamental|      Bolivia|         1953|
|    Justin Walton| Fundamental|      Bolivia|         1956|
|  Tiffany Parnell| Fundamental|      Bolivia|         1951|
|       Ralph Gray| Fundamental|    Argentina|         2005|
|      John Rusnak|    Superior|         Peru|         1986|
|     Nelson Bland|       Medio|       Guyana|         2010|
|     Michael Levi|       Medio|French Guiana|         1975|
|Susan Bridenbaker|    Superior|     Suriname|         1956|
|       Rosa Goode|    Superior|     Paraguay|         1993|
|      Edna Scharf|    Superior|      Uruguay|         1992|
|    Jason Trundle|    S

# Etapa 8

In [35]:
millenials_count = birth_date_df.select("*").where((dff.col("AnoNascimento") >= 1980) & (dff.col("AnoNascimento") <= 1994)).count()
print(millenials_count)

2276151


# Etapa 9

In [37]:
spark.sql("SELECT COUNT(*) FROM People WHERE AnoNascimento BETWEEN 1980 AND  1994").show()

+--------+
|count(1)|
+--------+
| 2276151|
+--------+



# Etapa 10

In [49]:
generation_df = spark.sql("""
    WITH temp_table AS (
        SELECT Nomes, Pais,
            CASE
                WHEN AnoNascimento BETWEEN 1944 AND 1964 THEN 'BB'
                WHEN AnoNascimento BETWEEN 1965 AND 1979 THEN 'X'
                WHEN AnoNascimento BETWEEN 1980 AND 1994 THEN 'Y'
                WHEN AnoNascimento BETWEEN 1995 AND 2015 THEN 'Z'
                ELSE 'Undefined'
            END AS Geracao
        FROM People
    )
    SELECT Pais, Geracao, COUNT(*) AS Quantidade 
    FROM temp_table
    GROUP BY Pais, Geracao
""")

generation_df.sort("Pais", "Geracao", "Quantidade").show()

+---------+-------+----------+
|     Pais|Geracao|Quantidade|
+---------+-------+----------+
|Argentina|     BB|    231901|
|Argentina|      X|    175737|
|Argentina|      Y|    175407|
|Argentina|      Z|    186737|
|  Bolivia|     BB|    233970|
|  Bolivia|      X|    175285|
|  Bolivia|      Y|    171186|
|  Bolivia|      Z|    186538|
|   Brazil|     BB|    236166|
|   Brazil|      X|    175088|
|   Brazil|      Y|    173095|
|   Brazil|      Z|    183610|
|    Chile|     BB|    233104|
|    Chile|      X|    173080|
|    Chile|      Y|    175085|
|    Chile|      Z|    186230|
| Colombia|     BB|    235138|
| Colombia|      X|    177001|
| Colombia|      Y|    174849|
| Colombia|      Z|    187093|
+---------+-------+----------+
only showing top 20 rows

